In [2]:
import sklearn.feature_extraction.text as txt
from sklearn import svm
from joblib import dump
from snowflake.snowpark import functions as fn
from snowflake.snowpark.session import Session
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions

In [ ]:
connection_parameters = {
    "account": "xyz",
    "user": "xyz",
    "password": "xyz",
    "role": "accountadmin",
    "warehouse": "compute_wh",
    "database": "test",
    "schema": "public"
}

session = Session.builder.configs(connection_parameters).create()

In [10]:
session.sql("""list   @yelp_stage;""").collect()

[Row(name='s3://namastesqlsid/yelp_project/split_file_1.json', size=536103795, md5='53b0c22d9d7db9906e4665568df1141b-64', last_modified='Sun, 23 Mar 2025 05:45:08 GMT'),
 Row(name='s3://namastesqlsid/yelp_project/split_file_10.json', size=533360193, md5='6f55e6a493c7e19e294ce18448851c88-64', last_modified='Sun, 23 Mar 2025 05:48:12 GMT'),
 Row(name='s3://namastesqlsid/yelp_project/split_file_2.json', size=533397515, md5='ad1abb5ef16089a6113e470be819a994-64', last_modified='Sun, 23 Mar 2025 05:51:18 GMT'),
 Row(name='s3://namastesqlsid/yelp_project/split_file_3.json', size=531352098, md5='c0a434380a4534bed4f949f9ef61c9fd-64', last_modified='Sun, 23 Mar 2025 05:54:22 GMT'),
 Row(name='s3://namastesqlsid/yelp_project/split_file_4.json', size=538918564, md5='8c7d12bba05571693606b5edb5f85e11-65', last_modified='Sun, 23 Mar 2025 05:57:25 GMT'),
 Row(name='s3://namastesqlsid/yelp_project/split_file_5.json', size=538669607, md5='ada9b6fdc3cb3e2694e5ffaac52a4f7a-65', last_modified='Sun, 23 Mar 

In [14]:
df=session.read.option("infer_schema", True).json("@yelp_stage")
df.show(5)

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"business_id"           |"cool"  |"date"               |"funny"  |"review_id"             |"stars"  |"text"                                              |"useful"  |"user_id"               |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|XQfwVwDr-v0ZS3_CbbE5Xw  |0       |2018-07-07 22:09:11  |0        |KU_O5udG6zpxOg-VcAEodg  |3.0      |If you decide to eat here, just be aware it is ...  |0         |mh_-eMZ6K5RLWhZyISBhwA  |
|                        |        |                     |         |                        |         |                                                    |          |                        |
|                        |        |     

In [17]:
df.schema

StructType([StructField('"business_id"', StringType(), nullable=True), StructField('"cool"', LongType(), nullable=True), StructField('"date"', TimestampType(tz=ntz), nullable=True), StructField('"funny"', LongType(), nullable=True), StructField('"review_id"', StringType(), nullable=True), StructField('"stars"', DecimalType(2, 1), nullable=True), StructField('"text"', StringType(), nullable=True), StructField('"useful"', LongType(), nullable=True), StructField('"user_id"', StringType(), nullable=True)])

In [ ]:
from snowflake.snowpark.functions import col

# Rename columns by removing double quotes
cleaned_columns = [col(c).alias(c.replace('"', '')) for c in df.columns] # creating aliases

# Create a new DataFrame with cleaned column names
cleaned_df = df.select(*cleaned_columns)

# Display the DataFrame with cleaned column names
cleaned_df.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"BUSINESS_ID"           |"COOL"  |"DATE"               |"FUNNY"  |"REVIEW_ID"             |"STARS"  |"TEXT"                                              |"USEFUL"  |"USER_ID"               |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|Al8y5FTs9tW5VsxTVy9ZMg  |0       |2020-12-01 17:44:08  |0        |QbIWNp2ZXR8NkeZNL7sl2g  |5.0      |Alyans                                              |1         |Z14xeXGKzjrj0N2rYMKoSg  |
|                        |        |                     |         |                        |         |                                                    |          |                        |
|                        |        |     

In [22]:
cleaned_df.where(fn.col("funny") == 1.0).show(5)

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"BUSINESS_ID"           |"COOL"  |"DATE"               |"FUNNY"  |"REVIEW_ID"             |"STARS"  |"TEXT"                                              |"USEFUL"  |"USER_ID"               |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|PVRs2-0uQYqfdhoCidwA_w  |3       |2016-07-15 02:49:58  |1        |B0A0fFR064pIDnc2cCnMJw  |4.0      |My husband - a meat and potatoes guy - was in t...  |4         |3FNCakIRyM0zjLTdsJNsNQ  |
|                        |        |                     |         |                        |         |                                                    |          |                        |
|                        |        |     

In [24]:
#will put another json file in an interal stage
session.sql("""CREATE STAGE yelp_business file_format = (TYPE = 'JSON' ) """).collect()

[Row(status='Stage area YELP_BUSINESS successfully created.')]

In [25]:
session.file.put('yelp_academic_dataset_business.json','yelp_business')

[PutResult(source='yelp_academic_dataset_business.json', target='yelp_academic_dataset_business.json.gz', source_size=118863795, target_size=20340592, source_compression='NONE', target_compression='GZIP', status='UPLOADED', message='')]

In [27]:
df_business=session.read.option("infer_schema",True).json("@yelp_business")

In [28]:
df_business.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"address"                                   |"attributes"                                        |"business_id"           |"categories"                                        |"city"          |"hours"                     |"is_open"  |"latitude"     |"longitude"      |"name"                    |"postal_code"  |"review_count"  |"stars"  |"state"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [45]:
# Rename columns by removing double quotes
print(df_business.columns)
col_list=[]
for i in df_business.columns:
    new_col=i.replace('"','')
    col_list.append(new_col)
    df_business=df_business.with_column_renamed(i,new_col)

df_business.columns



['ADDRESS', 'ATTRIBUTES', 'BUSINESS_ID', 'CATEGORIES', 'CITY', 'HOURS', 'IS_OPEN', 'LATITUDE', 'LONGITUDE', 'NAME', 'POSTAL_CODE', 'REVIEW_COUNT', 'STARS', 'STATE']


['ADDRESS',
 'ATTRIBUTES',
 'BUSINESS_ID',
 'CATEGORIES',
 'CITY',
 'HOURS',
 'IS_OPEN',
 'LATITUDE',
 'LONGITUDE',
 'NAME',
 'POSTAL_CODE',
 'REVIEW_COUNT',
 'STARS',
 'STATE']

In [47]:
df_business.select(col('address')).collect()

[Row(ADDRESS='1616 Chapala St, Ste 2'),
 Row(ADDRESS='87 Grasso Plaza Shopping Center'),
 Row(ADDRESS='5255 E Broadway Blvd'),
 Row(ADDRESS='935 Race St'),
 Row(ADDRESS='101 Walnut St'),
 Row(ADDRESS='615 S Main St'),
 Row(ADDRESS='8522 Eager Road, Dierbergs Brentwood Point'),
 Row(ADDRESS='400 Pasadena Ave S'),
 Row(ADDRESS='8025 Mackenzie Rd'),
 Row(ADDRESS='2312 Dickerson Pike'),
 Row(ADDRESS='21705 Village Lakes Sc Dr'),
 Row(ADDRESS=''),
 Row(ADDRESS='8901 US 31 S'),
 Row(ADDRESS='15 N Missouri Ave'),
 Row(ADDRESS='2575 E Bay Dr'),
 Row(ADDRESS='205 Race St'),
 Row(ADDRESS='625 N Stone Ave'),
 Row(ADDRESS='712 Adams St'),
 Row(ADDRESS='1241 Airline Dr'),
 Row(ADDRESS='1224 South St'),
 Row(ADDRESS='10359 104 Street NW'),
 Row(ADDRESS='8675 River Crossing Blvd'),
 Row(ADDRESS='6625 E 82nd St'),
 Row(ADDRESS='5505 S Virginia St'),
 Row(ADDRESS='1160 Gallatin Pike S'),
 Row(ADDRESS='662 Durham Rd'),
 Row(ADDRESS='827-833 State St'),
 Row(ADDRESS='215 1st Ave S'),
 Row(ADDRESS='767 S 

In [59]:
df_business.show(5)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ADDRESS"                        |"ATTRIBUTES"                                        |"BUSINESS_ID"           |"CATEGORIES"                                        |"CITY"         |"HOURS"                     |"IS_OPEN"  |"LATITUDE"     |"LONGITUDE"      |"NAME"                    |"POSTAL_CODE"  |"REVIEW_COUNT"  |"STARS"  |"STATE"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# find  categories that have most number businesses registered with 
task1=df_business.select(col('business_id'),fn.split(col('categories'),fn.lit(',')).alias('categories_list')).with_column("category",fn.explode("categories_list")).select(col("business_id"),fn.trim(col("category")).alias("category"))
task1.groupBy(col('category')).agg(fn.count("*").alias("total_businesses")).sort(col('total_businesses').desc()).show()

-----------------------------------------
|"CATEGORY"        |"TOTAL_BUSINESSES"  |
-----------------------------------------
|Restaurants       |52268               |
|Food              |27781               |
|Shopping          |24395               |
|Home Services     |14356               |
|Beauty & Spas     |14292               |
|Nightlife         |12281               |
|Health & Medical  |11890               |
|Local Services    |11198               |
|Bars              |11065               |
|Automotive        |10773               |
-----------------------------------------



In [82]:
# find top 10 users who have reviewd the most restaurants catrogory of business
business_restaurant_df= df_business.filter(fn.lower(col("categories")).like("%restaurants%")).select(col("business_id"),col("categories"))
business_restaurant_df.show()
task2=cleaned_df.select(col("user_id"),col("business_id")).join(business_restaurant_df,on="business_id",how="inner")
task2.show()

-------------------------------------------------------------------------------
|"BUSINESS_ID"           |"CATEGORIES"                                        |
-------------------------------------------------------------------------------
|MTSW4McQd7CbVtyjqoe9mw  |Restaurants, Food, Bubble Tea, Coffee & Tea, Ba...  |
|CF33F8-E6oudUQ46HnavjQ  |Burgers, Fast Food, Sandwiches, Food, Ice Cream...  |
|k0hlBqXX-Bt0vf1op7Jr1w  |Pubs, Restaurants, Italian, Bars, American (Tra...  |
|bBDDEgkFA1Otx9Lfe7BZUQ  |Ice Cream & Frozen Yogurt, Fast Food, Burgers, ...  |
|eEOYSgkmpB90uNA7lDOMRA  |Vietnamese, Food, Restaurants, Food Trucks          |
|il_Ro8jwPlHresjw9EGmBg  |American (Traditional), Restaurants, Diners, Br...  |
|0bPLkL0QhhPO5kt1_EXmNQ  |Food, Delis, Italian, Bakeries, Restaurants         |
|MUTTqe8uqyMdBl186RmNeA  |Sushi Bars, Restaurants, Japanese                   |
|ROeacJQwBeh05Rqg7F6TCg  |Korean, Restaurants                                 |
|WKMJwqnfZKsAae75RMP6jA  |Coffee & Tea, 

In [90]:
task2.group_by(fn.col('user_id')).agg(fn.count_distinct(fn.col('business_id')).alias('no_of_business_reviewed')).sort(col('no_of_business_reviewed').desc()).show()

------------------------------------------------------
|"USER_ID"               |"NO_OF_BUSINESS_REVIEWED"  |
------------------------------------------------------
|-G7Zkl1wIWBBmD0KRy_sCw  |1202                       |
|_BcWyKQL16ndpBdggh2kNA  |1166                       |
|fr1Hz2acAb3OaL3l6DyKNg  |1058                       |
|1HM81n6n4iPIFU5d2Lokhw  |1009                       |
|Xw7ZjaGfr0WNVt6s_5KZfA  |926                        |
|ET8n-r7glWYqZhuR6GcdNw  |891                        |
|pou3BbKsIozfH50rxmnMew  |849                        |
|VL12EhEdT4OWqGq0nIqkzw  |832                        |
|wXdbkFZsfDR7utJvbWElyA  |773                        |
|ouODopBKF3AqfCkuQEnrDg  |744                        |
------------------------------------------------------



In [92]:
#3 number of reviews per category of business
task1.join(cleaned_df.select_expr('business_id','review_id'),on="business_id",how="inner").group_by(col('category')).agg(fn.count('*').alias('review_count')).sort(col('review_count').desc()).show()

----------------------------------------------
|"CATEGORY"                 |"REVIEW_COUNT"  |
----------------------------------------------
|Restaurants                |4724471         |
|Food                       |1813593         |
|Nightlife                  |1539757         |
|Bars                       |1455553         |
|American (Traditional)     |1011646         |
|American (New)             |984540          |
|Breakfast & Brunch         |867430          |
|Sandwiches                 |691864          |
|Seafood                    |620247          |
|Event Planning & Services  |609553          |
----------------------------------------------



In [93]:
cleaned_df.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"BUSINESS_ID"           |"COOL"  |"DATE"               |"FUNNY"  |"REVIEW_ID"             |"STARS"  |"TEXT"                                              |"USEFUL"  |"USER_ID"               |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|iRIHK8-EwpeffwvoO4nzIA  |0       |2018-10-04 16:39:12  |0        |oXpWjOG2rue-2bHbjvDZIQ  |5.0      |Awesome spot. This is one of my favorite places...  |0         |1uHAAjOX18Px4OibCMI3pg  |
|U9TJ13_B6mySAb1RI4lhug  |0       |2017-05-13 20:40:04  |0        |QVL6hH7M0nv5ljHCjRfK0w  |3.0      |I like Rumor, but some nights, Idk why they cha...  |0         |KZXmUq6y_0k4guwAfPmQcQ  |
|oBNrLz4EDhiscSlbOl8uAw  |0       |2018-

In [94]:
from snowflake.snowpark.window import Window
cleaned_df.select(col('business_id'),col('review_id'),col('text'),fn.row_number().over(Window.partition_by(col("business_id")).order_by(col("date").desc())).alias("latest")).filter(col('latest') < 4).show()

-------------------------------------------------------------------------------------------------------------------
|"BUSINESS_ID"           |"REVIEW_ID"             |"TEXT"                                              |"LATEST"  |
-------------------------------------------------------------------------------------------------------------------
|bZ9y54ian0Uin6nar0VkcQ  |fOB9uP_NIc4RHPGNjegrpw  |Crystal was amazing at Eye Encounters! Very hel...  |1         |
|bZ9y54ian0Uin6nar0VkcQ  |60qvvdqQMtIMoEpM0HTiLA  |I went to get new reading glasses because the o...  |2         |
|bZ9y54ian0Uin6nar0VkcQ  |-OdOHbit5FlgDGFneZcSJg  |My whole experience here was great doctor and e...  |3         |
|bZDb2uUJcDh3wWIiWLU_gA  |h_3sgShWl3mVfrtjskdb8A  |The service at this particular location was alw...  |1         |
|bZDb2uUJcDh3wWIiWLU_gA  |Hrl4M0DHlXp0H6-XHFbfww  |Normally Im not into rest stop food but.. Im a ...  |2         |
|bZDb2uUJcDh3wWIiWLU_gA  |A_3N9HfUcm1xGuNgJPg3mA  |Service is horrid! A 

In [96]:
task5=cleaned_df.with_column('month',fn.month(col('date'))).group_by(col('month')).agg(fn.count('*').alias('reviews_per_month')).sort(col('reviews_per_month').desc())
task5.show()

---------------------------------
|"MONTH"  |"REVIEWS_PER_MONTH"  |
---------------------------------
|7        |654627               |
|8        |636384               |
|1        |604532               |
|6        |601737               |
|3        |598555               |
|5        |586575               |
|10       |571809               |
|9        |565374               |
|4        |551471               |
|2        |544125               |
---------------------------------



In [ ]:
task6=cleaned_df.join(df_business.with_column_renamed(fn.col('stars'),'total_stars'),on="business_id",how='inner').group_by('business_id').agg(fn.count('*').alias('total_reviews'),fn.count(fn.when(col("stars")==5.0, 1)).alias("five_star_review_count"))


-----------------------------------------------------------------------
|"BUSINESS_ID"           |"TOTAL_REVIEWS"  |"FIVE_STAR_REVIEW_COUNT"  |
-----------------------------------------------------------------------
|ws-gbVjt56zCQC-CR5pjTQ  |33               |10                        |
|DxqCwF6sMDVfae7hc3HS8Q  |9                |7                         |
|CcnlqyP_asIREYktlsAuIA  |11               |11                        |
|FuU-wjbEslczzWAQzht50g  |6                |5                         |
|9kCutyCm1b8UuCRJF3Hw5A  |5                |3                         |
|l9mfP_p3LetJNM9V0XKmkw  |7                |0                         |
|wghnIlMb_i5U46HMBGx9ig  |23               |5                         |
|5LbiHS5xbeKUxd9ulIJz4w  |21               |5                         |
|Azwby99MXxgB8bjaCwXkKg  |5                |1                         |
|9Vh8_qHO1lwZ4a7RB2BkWg  |81               |18                        |
----------------------------------------------------------------

In [107]:
task6.with_column('5_star_percentage',fn.col("five_star_review_count") / fn.col("total_reviews")).sort(fn.col('5_star_percentage')).show()


---------------------------------------------------------------------------------------------
|"BUSINESS_ID"           |"TOTAL_REVIEWS"  |"FIVE_STAR_REVIEW_COUNT"  |"5_star_percentage"  |
---------------------------------------------------------------------------------------------
|OL9CgTWy7rU2eNq6F13JBg  |14               |0                         |0.000000             |
|BYM9GFFZaEODMFQ2G5141g  |7                |0                         |0.000000             |
|c6xp1swmd-_P87kt3xggDA  |16               |0                         |0.000000             |
|SYbznwoCPzIRA55kZ865jQ  |15               |0                         |0.000000             |
|PF0zJt5Oc_uQ4UlupWrMeg  |5                |0                         |0.000000             |
|Brd3LK9-rH3eEQuzTD71Ug  |5                |0                         |0.000000             |
|ByHCHgbGqycKnz21DF3v9w  |6                |0                         |0.000000             |
|smYrzUcVgdo6z4-LWqqB-w  |35               |0               

In [ ]:
#3 number of reviews per category of business
task1.join(cleaned_df.select_expr('business_id','review_id'),on="business_id",how="inner").group_by(col('category')).agg(fn.count('*').alias('review_count')).sort(col('review_count').desc()).show()

In [109]:
cleaned_df.group_by('business_id').agg(fn.count('*').alias('review_count'),fn.avg(col('stars')).alias('avg_rating')).where(col('review_count') >=100).sort(col('avg_rating').desc()).select(col('business_id'),col('avg_rating')).show()

-----------------------------------------
|"BUSINESS_ID"           |"AVG_RATING"  |
-----------------------------------------
|-siOxQQcGKtb-04dX0Cxnw  |5.0000000     |
|1RqfozJoosHAsKZhc5PY7w  |5.0000000     |
|NDwoKO79_T49UEKVDlHd3A  |4.9835616     |
|PmYILphI46sDCAxkDZARjQ  |4.9781022     |
|ak-RiYgUZzM_90c8ONWFzw  |4.9775281     |
|QNilrbTi8912ye2ztnBMpA  |4.9734043     |
|DboqYyH-S8pV6WxaF9Plow  |4.9729730     |
|8q25GCkL0PngPIlCu2j4Vw  |4.9712230     |
|L4JFBsgfLZIL9SJ5uxXgJw  |4.9711538     |
|im3hUe2nigm2Xm-Z1SNXIg  |4.9702970     |
-----------------------------------------



In [1]:
!pip  install nbconvert[webpdf]

     ---------------------------------------- 83.4/83.4 kB ? eta 0:00:00
     -------------------------------------- 101.4/101.4 kB 6.1 MB/s eta 0:00:00
